In [30]:
%load_ext autoreload
%autoreload 2

#model and 
from utils import *
X,labels=read_data('datasets/data sets/MSRC-v1.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()
from models import MSE
model_MSE = MSE()
Y = model_MSE.mse(X,gamma = 0.9,d_=10,t=1e8,epoch=500)

nmi_list = []
for i in range(50):
    pred = kmeans(Y,7)
    # criterion
    from sklearn.metrics.cluster import normalized_mutual_info_score
    nmi = normalized_mutual_info_score(labels,pred)
    nmi_list.append(nmi)
print(np.mean(nmi_list))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


d:\Soft\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Soft\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Soft\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
d:\Soft\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available thre

0.4642928616874672
